In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import tkinter

In [3]:
from tkinter import messagebox

In [4]:
import smtplib

In [5]:
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [6]:
root = tkinter.Tk()

In [7]:
root.withdraw()

''

In [8]:
# root.title("Access Denied")

In [9]:
from keras.models import load_model

In [10]:
model = load_model('Face_Mask_Predictor.h1')

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 150, 150, 32)      320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 37, 37, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 18, 18, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 18, 18, 128)      

In [12]:
from mtcnn import MTCNN

In [13]:
detector = MTCNN()

In [14]:
labels={1:'MASK',0:'NO MASK', 2:'NO FACE FOUND'}
color={1:(0,255,0),0:(0,0,255)}

In [15]:
SUBJECT = "Subject"
TEXT = "One Visiter violeted Face Mask Policy. See in the camera to recognize the user. A Person has been detected without mask"

In [ ]:
video = cv2.VideoCapture(0)
video.set(cv2.CAP_PROP_FRAME_WIDTH, 1000)
video.set(cv2.CAP_PROP_FRAME_HEIGHT, 1000)
while True:
    _ , img = video.read()
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    faces = detector.detect_faces(img_rgb)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    for face in faces:
        (x,y,w,h) = face['box']
        face_img = gray_img[y:y+h, x:x+w]
        resized_img = cv2.resize(face_img, (150,150))
        reshaped_img = np.reshape(resized_img, (1,150,150,1))
        result = int(model.predict(reshaped_img)[0][0])
        cv2.rectangle(img,(x,y),(x+w,y+h),color[result],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color[result],-1)
        cv2.putText(img, labels[result], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        if(result == 0):
            messagebox.showwarning("Warning","Access Denied .Please wear mask")
            
            message = 'Subject: {}\n\n{}'.format(SUBJECT,TEXT)
            mail = smtplib.SMTP('smtp.gmail.com', 587)
            mail.ehlo()
            mail.starttls()
            mail.login('dhyey1974@gmail.com','joshi2612')
            mail.sendmail('dhyey1974@gmail.com','dhyey1974@gmail.com',TEXT)
            mail.close
            
#             email = 'dhyey1974@gmail.com'
#             password = 'joshi2612'
#             send_to_email = 'dhyey1974@gmail@gmail.com'
#             subject = 'This is the subject' # The subject line
#             message = 'This is my message from dhyey .Please wear mask'

#             msg = MIMEMultipart()
#             msg['From'] = email
#             msg['To'] = send_to_email
#             msg['Subject'] = subject

             
#             msg.attach(MIMEText(message, 'plain'))

#             server = smtplib.SMTP('smtp.gmail.com', 587)
#             server.starttls()
#             server.login(email, password)
#             text = msg.as_string() 
#             server.sendmail(email, send_to_email, text)
#             server.quit()
        else:
            pass
            break
    cv2.imshow('LIVE', img)
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()
video.release()